In [2]:
!pip install boto3

   ---------------------------------------- 0.0/139.3 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.3 kB 1.4 MB/s eta 0:00:01
   -------------------------- ------------- 92.2/139.3 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 139.3/139.3 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.3 MB 7.5 MB/s eta 0:00:02
   - -------------------------------------- 0.3/12.3 MB 4.3 MB/s eta 0:00:03
   - -------------------------------------- 0.5/12.3 MB 4.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/12.3 MB 4.5 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/12.3 MB 4.4 MB/s eta 0:00:03
   --- ------------------------------------ 1.2/12.3 MB 4.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.4/12.3 MB 4.5 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/12.3 MB 4.6 MB/s eta 0:00:03
   --

In [ ]:
# documentation :
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#rekognition

In [ ]:
# for "start segmnent_detection" : 
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition/client/start_segment_detection.html#

In Amazon Rekognition, start segmentation and get segmentation are operations used for video analysis. They are part of the asynchronous video processing capabilities in Amazon Rekognition, which allows you to analyze and retrieve results from video files

Start Segmentation

StartSegmentDetection is an operation that starts the asynchronous detection of segments in a stored video. Segments can include different types of scenes such as shots, black frames, color bars, end credits, and more. This operation is typically used to analyze long-form content like movies, TV shows, or sports events to detect changes and structure within the video.
Key Points:

    Initiate Analysis: It initiates the analysis of the video stored in an Amazon S3 bucket.
    Asynchronous Operation: The operation runs asynchronously, meaning you initiate it and then periodically check for results.
    Notification: You can specify an Amazon SNS topic to receive notifications when the segment detection is complete.
    Parameters: You can specify the types of segments you are interested in, such as shot detection or black frame detection.

In [8]:
# Print out each queue name, which is part of its ARN
for queue in sqs.queues.all():
    print(queue.url)

https://sqs.ap-south-1.amazonaws.com/381492233747/shot_detection
https://sqs.ap-south-1.amazonaws.com/381492233747/test


In [9]:
import boto3
# retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  vivekp


In [ ]:
#documentation code:

'''
response = client.start_segment_detection(
    Video={
        'S3Object': {
            'Bucket': 'string',
            'Name': 'string',
            'Version': 'string'
        }
    },
    ClientRequestToken='string',
    NotificationChannel={
        'SNSTopicArn': 'string',
        'RoleArn': 'string'
    },
    JobTag='string',
    Filters={
        'TechnicalCueFilter': {
            'MinSegmentConfidence': ...,
            'BlackFrame': {
                'MaxPixelThreshold': ...,
                'MinCoveragePercentage': ...
            }
        },
        'ShotFilter': {
            'MinSegmentConfidence': ...
        }
    },
    SegmentTypes=[
        'TECHNICAL_CUE'|'SHOT',
    ]
)
'''

In [11]:
import boto3

# now we intialize the boto
rekognition_client = boto3.client('rekognition')

# now parameters for the start_segment_detection call
video = {
    'S3Object': {
        'Bucket': 'vivekp',        #give bucket name
        'Name': '1 minute funny videos.mp4'    #our video name
    }
}

notification_channel = {
    'SNSTopicArn': 'arn:aws:sns:ap-south-1:381492233747:shot_detect',       #ur SNSs
    'RoleArn': 'arn:aws:iam::381492233747:role/vivek_role'                  #ur role
}

filters = {
    'TechnicalCueFilter': {
        'MinSegmentConfidence': 70.0,
        'BlackFrame': {
            'MaxPixelThreshold': 0.1,
            'MinCoveragePercentage': 99.0
        }
    },
    'ShotFilter': {
        'MinSegmentConfidence': 70.0
    }
}

segment_types = ['SHOT']

# now starting segment detection
response = rekognition_client.start_segment_detection(
    Video=video,
    NotificationChannel=notification_channel,
    Filters=filters,
    SegmentTypes=segment_types
)

# prinitng the JobId 
print('segment detection job started with JobId:', response['JobId'])


segment detection job started with JobId: 59eeb125cf3d10e64eccc4c160fba9edea638ace7c876761fabd93be3159b499


Job id is an Job identifier

In the context of Amazon Rekognition, the JobId is a unique identifier assigned to each asynchronous job when it is started. This identifier is used to track the job's progress, check its status, and retrieve the results once the job is complete